# 1. Twitter Data

This notebook covers the code for downloading data from Twitter using official Twitter API. Few pre-requisites before downloading the data -

1. An active Twitter account
2. Raise a request for developer account for reseach project with Twitter and get it approved. Access to this account provides a download limit of 10,000,000 tweets per month using both v1 and v2 endpoint (Beta)

#### Import required libraries

In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from Twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
### Copy the bearer code from Twitter developer account
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAMjIRQEAAAAAfDKY0uUXk4Lc2EmTbXhDfGRG4QY%3DQYq0ZoS7ucmz4Zr7RCVc47spywTwlzXDYs3MrGCAMAb1R3JAAU'

In [3]:
### function to authorise using bearer token
def auth():
    return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
# function to save required fields when sending request to twitter
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id,referenced_tweets.id,attachments.poll_ids,attachments.media_keys',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source,attachments',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified,location',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'poll.fields':'duration_minutes,end_datetime,id,options,voting_status',
                    'media.fields':'media_key,type,duration_ms,height,preview_image_url,public_metrics,url,width',
                    'next_token': {}}
    return (search_url, query_params)


In [6]:
# connect to v2 endpoint API
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

#### Define csv format of the tweets downloaded

This is required as the output from twitter is in json format and these functions flatten the output in a csv file

In [7]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        try:
            if ('geo' in tweet):   
                geo = tweet['geo']['place_id']
            else:
                geo = " "
        except:
            geo: " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        #9.referenced tweet
        try:
            for tweet_ref in tweet['referenced_tweets']:
                referenced_tweets_id=tweet_ref['id']
                referenced_tweets_type=tweet_ref['type']
        except:
            referenced_tweets_id = 0
            referenced_tweets_type =''
        
        #10.media
        try:
            media = tweet['attachments']['media_keys'][0]
        except:
            media=''
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text,referenced_tweets_id,referenced_tweets_type,media ]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [8]:
def append_to_csv_users(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for user in json_response['includes']['users']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = user['id']

        # 2. Time created
        created_at = dateutil.parser.parse(user['created_at'])

        # 3. Location 
        try:
            location = user['location']
        except:
            location=''

        #4. verifie status
        verified = user['verified']
        
        # 5. username
        username = user['username']

        # 6. Tweet metrics
        followers_count = user['public_metrics']['followers_count']
        following_count = user['public_metrics']['following_count']
        listed_count = user['public_metrics']['listed_count']
        tweet_count = user['public_metrics']['tweet_count']

        # 7. source
        name = user['name']

        # 8. Tweet text
        description = user['description']
        
        
        
        # Assemble all data in a list
        res = [author_id, created_at, location, verified, username, followers_count, following_count, listed_count, tweet_count, name, description ]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of users added from this response: ", counter) 

In [9]:
def append_to_csv_tweet(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['includes']['tweets']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. reply settings
        reply_settings = tweet['reply_settings']

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        #9. conversation_id
        conversation_id = tweet['conversation_id']
        
        #10.media
        try:
            media = tweet['attachments']['media_keys'][0]
        except:
            media=''
        
        # Assemble all data in a list
        res = [author_id, created_at, reply_settings, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text,conversation_id, media ]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [10]:
def append_to_csv_media(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for media in json_response['includes']['media']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that
        

        # 1. media type
        media_type = media['type']


        # 2. media width
        width = media['width']

        # 3. media height
        height = media['height']

        # 4. url
        try:
            media_url = media['url']
        except:
            media_url=''
        
        # 5. media key
        media_key=media['media_key']

        # 6. duration metrics
        try:
            duration_ms = media['duration_ms']
        except:
            duration_ms =0
            
        #7. video image
        try:
            preview_image_url=media['preview_image_url']
        except:
            preview_image_url=''
        
        #8. video popularity
        try:
            view_count = media['public_metrics']['view_count']
        except:
            view_count=0     
        
        # Assemble all data in a list
        res = [media_key, media_type, media_url, width, height, duration_ms,view_count,preview_image_url ]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of media_keys added from this response: ", counter) 

In [12]:
# Create file
csvFile = open("tweet_data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet', 'referenced_tweets_id','referenced_tweets_type','media'])
csvFile.close()

#### for users
# Create file
csvFile = open("data_user.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author_id', 'created_at', 'location', 'verified', 'username', 'followers_count', 'following_count', 'listed_count', 'tweet_count', 'name', 'description' ])
csvFile.close()

### for tweets
# Create file
csvFile = open("tweet_data_ref.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author_id', 'created_at', 'reply_settings', 'tweet_id', 'lang', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source', 'text,conversation_id', 'media' ])
csvFile.close()

###for media
# Create file
csvFile = open("media_data_ref.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['media_key', 'media_type', 'media_url', 'width', 'height', 'duration_ms','view_count','preview_image_url'])
csvFile.close()

#### Twitter calls

Twitter has a limit of 500 tweets per request, so this code initiates a request with max_results = 500 and also, there is a flexibility - if request gets errored out, twitter generates token for next set of tweets indicating data for the searched keywords

In [16]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)

###SEARCH WORDS FOR BOTH BITCOIN & ETHEREUM
keyword = "#ETH OR $ETH OR #ETHERIUM lang:en -is:retweet"
#keyword = "#BTC OR $BTC OR #BITCOIN OR #btc lang:en -is:retweet"
start_list =    "2021-08-01T00:00:00.000Z"

end_list =      "2021-08-09T00:00:00.000Z"

max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Inputs
count = 0 # Counting tweets per time period
#max_count = 100 # Max tweets per time period
flag = True
next_token = None ###Replace the token if code gets interrupted and it will resume
    
# Check if flag is true
while flag:
    print("-------------------")
    print("Token: ", next_token)
    url = create_url(keyword, start_list,end_list, max_results)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
    # Save the token to use for next call
        next_token = json_response['meta']['next_token']
        print("Next Token: ", next_token)
        if result_count is not None and result_count > 0 and next_token is not None:
            #print("Start Date: ", start_list[i])
            append_to_csv(json_response, "tweet_data.csv")
            #append_to_csv(json_response, "data_user.csv")
            append_to_csv_users(json_response, "data_user.csv")
            append_to_csv_tweet(json_response, "tweet_data_ref.csv")
            append_to_csv_media(json_response, "media_data_ref.csv")
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(5)                
        # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            #print("Start Date: ", start_list[i])
            append_to_csv(json_response, "tweet_data.csv")
            #append_to_csv(json_response, "data_user.csv")
            append_to_csv_users(json_response, "data_user.csv")
            append_to_csv_tweet(json_response, "tweet_data_ref.csv")
            append_to_csv_media(json_response, "media_data_ref.csv")
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(5)
            
        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  b26v89c19zqg8o3fpdm7qhl669gztephzjnl63tha93st
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qhkvrh9sbksalht6m4rt5rtdp
# of Tweets added from this response:  471
# of users added from this response:  309
# of Tweets added from this response:  214
# of media_keys added from this response:  75
Total # of Tweets added:  471
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qhkvrh9sbksalht6m4rt5rtdp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qhkvohiqc6d724psacsq1bgxp
# of Tweets added from this response:  463
# of users added from this response:  332
# of Tweets added from this response:  211
# of media_keys added from this response:  47
Total # of Tweets added:  934
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qhkvohiqc6d724psacsq1bgxp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qhkvmxm8chtl042fvi94n7799
# of Tweets added from this response:  471
# of users added from this

# of media_keys added from this response:  48
Total # of Tweets added:  10642
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qhk9w1vf94hbveucwl6yrm7p9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qhk9uilzxtd4zqg3k2iq3cpkt
# of Tweets added from this response:  493
# of users added from this response:  410
# of Tweets added from this response:  225
# of media_keys added from this response:  37
Total # of Tweets added:  11135
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qhk9uilzxtd4zqg3k2iq3cpkt
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qhk9szrvgoup9j58fac1jpsot
# of Tweets added from this response:  489
# of users added from this response:  403
# of Tweets added from this response:  197
# of media_keys added from this response:  59
Total # of Tweets added:  11624
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qhk9szrvgoup9j58fac1jpsot
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8

-------------------
Token:  b26v89c19zqg8o3fpdm7qhjo9ohs5qtz8uvn8yxkwnh4t
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qhjo85nu8d8zjslyml76egnzx
# of Tweets added from this response:  488
# of users added from this response:  394
# of Tweets added from this response:  187
# of media_keys added from this response:  43
Total # of Tweets added:  21250
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qhjo85nu8d8zjslyml76egnzx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qfhnkbofojkjteyc48dxwsakd
# of Tweets added from this response:  479
# of users added from this response:  366
# of Tweets added from this response:  196
# of media_keys added from this response:  41
Total # of Tweets added:  21729
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qfhnkbofojkjteyc48dxwsakd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qfhnit9pczeyoonsd7kaoyohp
# of Tweets added from this response:  466
# of users added from 

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qfh1zhcu6oijp17jgxvx0go71
# of Tweets added from this response:  486
# of users added from this response:  435
# of Tweets added from this response:  209
# of media_keys added from this response:  43
Total # of Tweets added:  31487
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qfh1zhcu6oijp17jgxvx0go71
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qfh1xyb4gm8je8mcebjlbuqnx
# of Tweets added from this response:  481
# of users added from this response:  409
# of Tweets added from this response:  222
# of media_keys added from this response:  83
Total # of Tweets added:  31968
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qfh1xyb4gm8je8mcebjlbuqnx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qfh1wfgumf1d2agslwmixj0jh
# of Tweets added from this response:  489
# of users added from this response:  413
# of Tweets added from this response:  202
# of media_

-------------------
Token:  b26v89c19zqg8o3fpdm7qfggbk2scflqdziqih5md8xvh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qfgga2ir2l78iil7rogqak0hp
# of Tweets added from this response:  480
# of users added from this response:  476
# of Tweets added from this response:  105
# of media_keys added from this response:  26
Total # of Tweets added:  42220
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qfgga2ir2l78iil7rogqak0hp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qfgga1o6dmthr9mxgsnh17d6l
# of Tweets added from this response:  493
# of users added from this response:  488
# of Tweets added from this response:  70
# of media_keys added from this response:  20
Total # of Tweets added:  42713
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7qfgga1o6dmthr9mxgsnh17d6l
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7qfgg8ima5skmwacfv9skvnr0d
# of Tweets added from this response:  493
# of users added from t

...

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7q2x2hbkh8o8aah5lmy2ddvh
# of Tweets added from this response:  463
# of users added from this response:  349
# of Tweets added from this response:  168
# of media_keys added from this response:  18
Total # of Tweets added:  471530
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7b7q2x2hbkh8o8aah5lmy2ddvh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7q2x1768h8hq4xlroqldxmrh
# of Tweets added from this response:  456
# of users added from this response:  336
# of Tweets added from this response:  145
# of media_keys added from this response:  18
Total # of Tweets added:  471986
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7b7q2x1768h8hq4xlroqldxmrh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7q2vjfikucyf5v9ovq5d92f1
# of Tweets added from this response:  475
# of users added from this response:  352
# of Tweets added from this response:  148
# of medi

-------------------
Token:  b26v89c19zqg8o3fpdm7b7pryjjmdc2hzjcal0xeitbb1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7phjseukql735ytmureey2yl
# of Tweets added from this response:  474
# of users added from this response:  376
# of Tweets added from this response:  155
# of media_keys added from this response:  19
Total # of Tweets added:  482006
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7b7phjseukql735ytmureey2yl
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7phia7zaoa69ighbeeuhs03h
# of Tweets added from this response:  468
# of users added from this response:  408
# of Tweets added from this response:  174
# of media_keys added from this response:  18
Total # of Tweets added:  482474
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7b7phia7zaoa69ighbeeuhs03h
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7phi8xtzaa13aroxtd71gwot
# of Tweets added from this response:  479
# of users added fro

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7p6i88quhjl7k9tv63insubh
# of Tweets added from this response:  483
# of users added from this response:  384
# of Tweets added from this response:  152
# of media_keys added from this response:  25
Total # of Tweets added:  492116
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7b7p6i88quhjl7k9tv63insubh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7p6gq1kr3naws3upsb4p8rr1
# of Tweets added from this response:  482
# of users added from this response:  349
# of Tweets added from this response:  180
# of media_keys added from this response:  15
Total # of Tweets added:  492598
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7b7p6gq1kr3naws3upsb4p8rr1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7p6f7uk2r7fey73e14pdhkl9
# of Tweets added from this response:  489
# of users added from this response:  335
# of Tweets added from this response:  160
# of medi

-------------------
Token:  b26v89c19zqg8o3fpdm7b7okyxti05e4dy4iyqms3yg71
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7okxf6yu8gi7xke6bt28635p
# of Tweets added from this response:  487
# of users added from this response:  352
# of Tweets added from this response:  163
# of media_keys added from this response:  20
Total # of Tweets added:  502709
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7b7okxf6yu8gi7xke6bt28635p
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7okvwd0vc4kpbkmh3wh0j1q5
# of Tweets added from this response:  485
# of users added from this response:  367
# of Tweets added from this response:  155
# of media_keys added from this response:  20
Total # of Tweets added:  503194
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdm7b7okvwd0vc4kpbkmh3wh0j1q5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdm7b7okue5z3spdt6xktlhjaloql
# of Tweets added from this response:  490
# of users added fro

KeyboardInterrupt: 

#### References

1. Twitter developer account - https://developer.twitter.com/en/apply-for-access
2. Twitter v2 endpoint fields list - https://developer.twitter.com/en/docs/twitter-api/fields
3. Twitter download script - https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a